In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from random import shuffle
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense


Using TensorFlow backend.


In [2]:
fer_path = '../Data/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013/fer2013.csv'
data = pd.read_csv(fer_path)

In [3]:
np.unique(data['Usage'], return_counts=True)

(array(['PrivateTest', 'PublicTest', 'Training'], dtype=object),
 array([ 3589,  3589, 28709]))

In [4]:
training_pixels = data[data['Usage'] == 'Training']['pixels'].tolist()
width, height = 48, 48
image_size = (width, height)
X_train = []
for pixel_sequence in training_pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    #     face = cv2.resize(face.astype('uint8'), image_size)
    X_train.append(face.astype('float32'))
    
X_train = np.asarray(X_train)
X_train = np.expand_dims(X_train, -1)
y_train = pd.get_dummies(data[data['Usage'] == 'Training']['emotion']).values

In [5]:
training_pixels = data[data['Usage'] == 'PublicTest']['pixels'].tolist()
width, height = 48, 48
image_size = (width, height)
X_test = []
for pixel_sequence in training_pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    #     face = cv2.resize(face.astype('uint8'), image_size)
    X_test.append(face.astype('float32'))
X_test = np.asarray(X_test)

X_test = np.expand_dims(X_test, -1)
y_test = pd.get_dummies(data[data['Usage'] == 'PublicTest']['emotion']).values

Faces shape type is (#records, image_width, image_height, 1)

In [8]:
IMG_SIZE = 48
model = Sequential()
model.add(Conv2D(64, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
# model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(7, activation = 'softmax'))

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=len(X_train)//20, epochs=20, verbose=1)

Epoch 1/20
28709/28709 [==============================] - 212s 7ms/step - loss: 1.7064 - acc: 0.3743
Epoch 2/20
28709/28709 [==============================] - 212s 7ms/step - loss: 1.2928 - acc: 0.5055
Epoch 3/20
28709/28709 [==============================] - 227s 8ms/step - loss: 1.1015 - acc: 0.5872
Epoch 4/20
28709/28709 [==============================] - 245s 9ms/step - loss: 0.9366 - acc: 0.6541
Epoch 5/20
28709/28709 [==============================] - 236s 8ms/step - loss: 0.7965 - acc: 0.7090
Epoch 6/20
28709/28709 [==============================] - 228s 8ms/step - loss: 0.6566 - acc: 0.7610
Epoch 7/20
28709/28709 [==============================] - 233s 8ms/step - loss: 0.5198 - acc: 0.8140
Epoch 8/20
28709/28709 [==============================] - 243s 8ms/step - loss: 0.4371 - acc: 0.8433
Epoch 9/20
28709/28709 [==============================] - 258s 9ms/step - loss: 0.3620 - acc: 0.8739
Epoch 10/20
28709/28709 [==============================] - 251s 9ms/step - loss: 0.3058 - a

In [10]:
model.save('keras_on_fer')

In [11]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Testing loss: {}, acc: {}\n'.format(loss, acc))

Testing loss: 2.360457666340762, acc: 0.5731401504680419

